# LIMPIEZA Y LDA

In [207]:
#IMPORTAR PAQUETES NECESARIOS.
import re 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd

#MATRIX TÉRMINO-DOCUMENTO
from sklearn.feature_extraction.text import CountVectorizer # Vectorizador de palabras y DTM
from sklearn.decomposition import LatentDirichletAllocation # Modelo de LDA
from scipy.sparse import csr_matrix # Para tratar Sparse Matrix


#LDA
import pyLDAvis 
from pyLDAvis import sklearn as sklearnlda #EXPORTAR EL MODELO (VISUALIZACIÓN)

#WORDCLOUD
from wordcloud import WordCloud


#Explicación Log-Likelihood

In [143]:
#CAMBIAR AQUÍ EL ARCHIVO
df = pd.read_csv("final_db.csv")

## Visualización de algunos tweets

In [144]:
#VEAMOS ALGUNOS TWEETS:
for i in df["tweet"][10000:10010]:
    print(i)
    print("\n-------------------------------")

#EL TRABAJO DE LIMPIEZA ES BASTANTE

@realDonaldTrump are you watching your president like the rest of us?!
#bidenharis2020 
#JoeBiden 
#DonaldTrump

////////////////////////////////
#JoeBiden is a good man. #DonaldTrump is an evil man. Our choice Tuesday couldn't be clearer. Here's an early look at Saturday's MNT lead editorial.

https://t.co/aR2hwhge1z

////////////////////////////////
@DemocracyStorm @ShawnInArizona Thank You Lori
He is with AZ 🌵🇺🇸🌵
He is with the whole country
#JohnMcCain 
#JoeBiden https://t.co/YeAiPuXymT

////////////////////////////////
@sentientist 1) I don’t believe that you voted for #Biden 2) So you don’t believe in justice for #Palestine 3) they are voting against their own interests 4) this cartoon makes everything WORSE #BobAndSally suck

////////////////////////////////
.@realDonaldTrump may well be laying the groundwork for trying to present his own rival slate of PA electors in case #JoeBiden wins there, but unfortunately for him, the governor in PA is a Democrat, and the final call on a 

# LIMPIEZA 

In [181]:
#LIMPIEZA GENERAL
df.tweet=df.tweet.str.lower()  #mayusulas
df.tweet=df.tweet.str.replace('[,\.!?\-!?\n\)\(\r]', ' ') # Borro Puntuaciones
df.tweet=df.tweet.str.replace('[0-9]', ' ') # Quito números
df.tweet=df.tweet.str.replace('[^a-zA-Záéíóúñ]+', ' ') #CONSERVAR CARACTERES Y ACENTOS
df.tweet=df.tweet.str.replace('  +', ' ') #quito espacios innecesarios

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<ipython-input-181-d546939a7b93>:3: DeprecationWarning: invalid escape sequence \.
  df.tweet=df.tweet.str.replace('[,\.!?\-!?\n\)\(\r]', ' ') # Borro Puntuaciones


In [183]:
A = "spectatorindex lets first wait until all the lawsuits are finalized before corrupt pedo alzheimer racist biden will be president bidenkamala trump trumpcv "

In [184]:
re.findall('trump[A-z]*', A)

['trump', 'trumpcv']

In [ ]:
#ACRONIMOS IMPORTANTES
#keep america great. kag
#make america great again: maga
#black lives matter: blm
#grand old party: gop

In [182]:
#LIMPIEZA ESPECIFICA
df.tweet=df.tweet.str.replace('biden[A-z]*', '') 
df.tweet=df.tweet.str.replace('joe[A-z]*', '')
df.tweet=df.tweet.str.replace('donald[A-z]*', '')
df.tweet=df.tweet.str.replace('kamala[A-z]*', '')
df.tweet=df.tweet.str.replace('trump[A-z]*', '')
df.tweet=df.tweet.str.replace('president[A-z]*', '')
df.tweet=df.tweet.str.replace('potus[A-z]*', '')

In [185]:
#LIMPIEZA ESPECIFICA DE PALABRAS
esp = ["want", "cnn", "https", "harris", "will", "just", "vote vote", "vote early", "rally", "msnbc", "don"["yes", "amp", "did", "doing", "trying", "ll", "ve", "en", "el", "like", "en", "el", "que", "elecciones", "la", "florida", "michigan", "best", "like", "gt"]
for i in esp:
    df.tweet=df.tweet.str.replace(i, '')


In [186]:
#VEAMOS ALGUNOS TWEETS CON LOs CAMBIOS:
for i in df["tweet"][0:1000]:
    print(i)
    print("\n------------------------------------------------------------------------------")

#EL TRABAJO DE LIMPIEZA ES BASTANTE

did the democrats not try to impeach over a phone call this matter is really shock camp confesses we not rule out possibility that met with burisma official pozharskyi t co qincynuswi

////////////////////////////////
 spectatorindex lets first wait until all the lawsuits are finalized before corrupt pedo alzheimer racist be 

////////////////////////////////
 johnwdean this would make me sleep a lot better john and i pray for it meanwhile  in case voteinperson in pa wi amp mi not by dropbox if able this reduces red mirage amp workload to count mail ins t co ey bdtsbvw

////////////////////////////////
of course this desperate man s certain that the magical mailinballots favoring him are legal while every of s aren t s a disgraced impeached term nothing more he no longer gets to live in our heads rent free bye demvoice t co bifnv rqdn t co i rrslvcpl

////////////////////////////////
i can t wait to have a and first lady with dignity again 

////////////////////////////////
nothing com

In [187]:
#LIMPIEZA DE STOPWORDS
# Ahora construiremos la matriz término-documento
n_vocab=1500 # máximo tamaño de vocabulario
tf_vectorizer = CountVectorizer(max_df=0.8, min_df=2, max_features=n_vocab, stop_words='english', ngram_range=(1,3)) # Al igual que un modelo, defino el objeto que construirá la matriz
tf = tf_vectorizer.fit_transform(df.tweet) # Aplico el objeto a un conjunto de textos
tf_feature_names = tf_vectorizer.get_feature_names() # Veo el vocabulario

In [203]:
num_topics=6 # Cuántos tópicos deseo
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=10,doc_topic_prior=0.1, topic_word_prior=0.1,random_state=23, verbose=1) # Construyo el objeto que es el modelo
lda.fit(tf) # Estimo el LDA

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(doc_topic_prior=0.1, n_components=6, random_state=23,
                          topic_word_prior=0.1, verbose=1)

In [204]:
# Construyo la función que me ayuda a ver las palabras más importantes de cada tópico
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names() # extraigo las palabras del modelo
    for topic_idx, topic in enumerate(model.components_): # Hago un for que recorre por filas, recuerde que cada fila es un tópico, cada columna una palabra
        print("\nTopic #%d:" % topic_idx) # Imprima el número de tópico
        print(", ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])) #Muestre las n palabras más importantes por orden 

In [205]:
print_topics(lda, tf_vectorizer, 15)


Topic #0:
hunter, debates, amp, debate, real, china, media, did, know, states, lies, family, said, people, twitter

Topic #1:
amp, vote, people, like, real, america, gop, country, make, election, know, let, time, years, going

Topic #2:
election, covid, coronavirus, electionresults, results, news, republican, democrats, democrat, election results, politics, gop, georgia, votes, ballots

Topic #3:
election, vote, elections, votes, win, electionday, amp, day, real, won, pa, voted, voting, today, count

Topic #4:
maga, real, landslide, kag, vote, blm, dump, voteblue, landslidevictory, bluewave, wtpblue, resist, america, wtpsenate, covid

Topic #5:
vote, election, la, en, florida, el, que, pennsylvania, michigan, real, elecciones, elections, electionday, arizona, electionnight


In [ ]:
##HAY QUE ELIMINAR TODAS LAS PALABRAS COMO JOE BIEN Y TRUMO Y SUS VARIANTES, KAMALA, HTTPS, DEBATES O CAMBIARLAS

In [107]:
%%time
# Juguemos con un hiper parámetro
likelihood=[]
values=[i for i in range(2,31,2)]
for i in values:
    modelo = LatentDirichletAllocation(n_components=i, max_iter=10,doc_topic_prior=0.1, topic_word_prior=0.1,random_state=23) # Construyo el objeto que es el modelo
    modelo.fit(tf)
    likelihood.append(modelo.score(tf))
    print(i)

2


KeyboardInterrupt: 

In [ ]:
# Visualizamos
plt.figure(figsize=(6,6))
plt.plot(values, likelihood)
plt.xlabel('Número de tópicos')
plt.ylabel('log-likelihood')

In [200]:
import pyLDAvis # Paquete que crea la visualización
from pyLDAvis import sklearn as sklearnlda
LDAvis_prepared=sklearnlda.prepare(lda, tf, tf_vectorizer ) # Preparo el modelo y sus resultados para la visualización

In [201]:
pyLDAvis.save_html(LDAvis_prepared, 'LDA.html') # Guardo la visualización como html